In [ ]:
# Inspect each page and if appropriate return details + sentiment

import requests
import pandas as pd
from bs4 import BeautifulSoup
from dateutil.parser import parse
from datetime import datetime, timezone
from functions import check_if_asx
from functions import named_entities
from functions import get_sentiment

asxCompanies = pd.read_csv('ASX_Listed_Companies.csv')
asxCompanies.drop('Listing date', axis = 1)

sentimentByCompany_new = pd.DataFrame(columns=["date", "title", "asxTag", "sentiment", "articleLength", "namedInTitle", 'author'])
matches = []
corpus = []

rawCorpus = pd.read_csv('scrape2_raw.csv')

for title in rawCorpus['title'].unique():

    print("Checking : " +str(title))
    try:
        article = rawCorpus[rawCorpus['title'] == title]
        author = article['author']
        time = article['date']
        paragraphs = article['paragraph']


        hasNe, toCheck, matchingCorpus = named_entities(title)
        if hasNe:
            hasAsx, asxTag, companyName, matchedEntity = check_if_asx(toCheck, asxCompanies)

            # If listed on ASX assign all paragraphs sentiment to company unless paragraph mentions another ASX company
            if hasAsx:
                print("found in title - Entity : " + str(matchedEntity) + " Matched to : " + str(asxTag) + " , " + str(companyName))
                paraCount = 0
                for paragraph in paragraphs:
                    paraCount += 1

                    # Check if paragraph has named entities and ASX Tag
                    hasNe, toCheck, matchingCorpus = named_entities(paragraph)
                    corpus.append(matchingCorpus)
                    hasAsx_para = False
                    if hasNe:
                        hasAsx_para, asxTag_para, companyName_para, matchedEntity_para = check_if_asx(toCheck, asxCompanies)

                    # If ASX Company found and differes from title Company return sentiment asociated with company found in paragraph
                    if hasAsx_para and asxTag_para != asxTag:
                        sentiment = get_sentiment(paragraph)
                        new_row = pd.DataFrame(
                            {"date": [time], "title": [title], "asxTag": [asxTag_para], "sentiment": [sentiment],
                             "articleLength": [len(paragraphs)], "namedInTitle": False, "authour": [author]})
                        sentimentByCompany_new = sentimentByCompany_new.append(new_row, ignore_index=True)
                        print("1 - Found in paragraph, overriding title - Entity : " + str(
                            matchedEntity_para) + " Matched to : " + str(asxTag_para) + " , " + str(companyName)
                              + " getting sentiment for : " + str(paraCount) + " of " + str(len(paragraphs)))
                        matches.append((matchedEntity_para, asxTag_para))

                    # otherwise return sentiment from paragraph assigned to company from title
                    else:
                        sentiment = get_sentiment(paragraph)
                        new_row = pd.DataFrame(
                            {"date": [time], "title": [title], "asxTag": [asxTag], "sentiment": [sentiment],
                             "articleLength": [len(paragraphs)], "namedInTitle": True, "authour": [author]})
                        sentimentByCompany_new = sentimentByCompany_new.append(new_row, ignore_index=True)
                        print("2 - From para : " + str(matchedEntity) + " Matched to : " + str(
                            asxTag) + " getting sentiment for : " + str(paraCount) + " of " + str(len(paragraphs)))
                        matches.append((matchedEntity, asxTag))

        # If no named entity in title check each paragraph
        elif not hasNe:

            # Iterate over all Paragraphs
            paraCount = 0
            for paragraph in paragraphs:
                paraCount += 1


                # Check if any named entities in paragraph
                hasNe, toCheck, matchingCorpus = named_entities(paragraph)
                corpus.append(matchingCorpus)
                if hasNe:

                    # If named entity present check if it is listed on ASX
                    hasAsx, asxTag, companyName, matchedEntity = check_if_asx(toCheck,asxCompanies)
                    # If ASX found find sentiment and return
                    if hasAsx:
                        sentiment = get_sentiment(paragraph)
                        new_row = pd.DataFrame(
                            {"date": [time], "title": [title], "asxTag": [asxTag], "sentiment": [sentiment],
                             "articleLength": [len(paragraphs)], "namedInTitle": False, "authour": [author]})
                        sentimentByCompany_new = sentimentByCompany_new.append(new_row, ignore_index=True)
                        print("4 - found in paragraph - Entity : " + str(matchedEntity) + " Matched to : " + str(
                            asxTag) + " getting sentiment for : " + str(paraCount) + " of " + str(len(paragraphs)))
                        matches.append((matchedEntity, asxTag))

                else:
                    print("Has no named entities : " + str(title) + " " + str(paraCount) + " of " + str(len(paragraphs)))
                    
    except:
        print("error with : " + str(title))

In [5]:
sentimentByCompany_new.to_csv('scrape2_sentiment.csv', index = False)


In [7]:
exportMatches = pd.DataFrame({"matches":matches})
exportMatches.to_csv('matches_2.csv', index = False)

DataTurks. (2018). Stanford CoreNLP: Training your own custom NER tagger. <br>
Retrieved from 
https://medium.com/swlh/stanford-corenlp-training-your-own-custom-ner-tagger-8119cc7dfc06 <br>
Gitau, C. (2018). Fuzzy String Matching. <br>
Retrieved from https://towardsdatascience.com/fuzzy-string-matching-in-python-68f240d910fe<br>
Godbole, N., Srinivasaiah, M., & Skiena, s. Large-Scale Sentiment Analysis for News and Blogs. 
Kaewphan, S., Hakala, K., Miekka, N., Salakoski, T., & Ginter, F. (2018). Wide-scope biomedical named entity recognition and normalization with CREs fuzzy matching and character level modeling. Database. doi:10.1093<br>
Magajna, T. Text Classification with State of the Art NLP Library — Flair.<br> Retrieved from https://towardsdatascience.com/text-classification-with-state-of-the-art-nlp-library-flair-b541d7add21f
